In [2]:
import tkinter as tk
from tkinter import ttk
from Crypto.PublicKey import RSA
from Crypto.Cipher import AES, PKCS1_OAEP
import base64
import os

def encrypt_message():
    message = entry.get()
    aes_key = os.urandom(16)

    # Encrypt message using AES
    encrypted_msg, nonce, tag = aes_encrypt(message, aes_key)

    # Encrypt AES key using RSA public key
    encrypted_aes_key = rsa_encrypt(aes_key, rsa_key.publickey())

    result_text.delete(1.0, tk.END)
    result_text.insert(tk.END, "Encrypted Message:\n" + base64.b64encode(encrypted_msg).decode('utf-8'))
    result_text.insert(tk.END, "\nAES Nonce:\n" + base64.b64encode(nonce).decode('utf-8'))
    result_text.insert(tk.END, "\nAES Tag:\n" + base64.b64encode(tag).decode('utf-8'))
    result_text.insert(tk.END, "\nEncrypted AES Key:\n" + base64.b64encode(encrypted_aes_key).decode('utf-8'))

def decrypt_message():
    encrypted_msg = base64.b64decode(result_text.get("1.0", tk.END).split('Encrypted Message:\n')[1].split('\nAES Nonce:')[0])
    nonce = base64.b64decode(result_text.get("1.0", tk.END).split('AES Nonce:\n')[1].split('\nAES Tag:')[0])
    tag = base64.b64decode(result_text.get("1.0", tk.END).split('AES Tag:\n')[1].split('\nEncrypted AES Key:')[0])
    encrypted_aes_key = base64.b64decode(result_text.get("1.0", tk.END).split('Encrypted AES Key:\n')[1])

    # Decrypt AES key using RSA private key
    decrypted_aes_key = rsa_decrypt(encrypted_aes_key, rsa_key)

    # Decrypt message using decrypted AES key
    decrypted_msg = aes_decrypt(encrypted_msg, nonce, tag, decrypted_aes_key)

    result_text.delete(1.0, tk.END)
    result_text.insert(tk.END, "Decrypted Message:\n" + decrypted_msg.decode('utf-8'))

def aes_encrypt(message, key):
    cipher = AES.new(key, AES.MODE_EAX)
    ciphertext, tag = cipher.encrypt_and_digest(message.encode('utf-8'))
    return ciphertext, cipher.nonce, tag

def aes_decrypt(ciphertext, nonce, tag, key):
    cipher = AES.new(key, AES.MODE_EAX, nonce=nonce)
    plaintext = cipher.decrypt_and_verify(ciphertext, tag)
    return plaintext

def rsa_encrypt(message, public_key):
    cipher = PKCS1_OAEP.new(public_key)
    ciphertext = cipher.encrypt(message)
    return ciphertext

def rsa_decrypt(ciphertext, private_key):
    cipher = PKCS1_OAEP.new(private_key)
    plaintext = cipher.decrypt(ciphertext)
    return plaintext  # Return as bytes

root = tk.Tk()
root.title("Hybrid System")

frame = tk.Frame(root)
frame.pack(padx=20, pady=20)

style = ttk.Style()
style.configure('TLabel', font=('calibri', 12, 'bold'))  # Label style

label = ttk.Label(frame, text="ENTER A MESSAGE:", style='TLabel')
label.pack()

entry = tk.Entry(frame, width=40)
entry.pack()

style.configure('TButton', font=('calibri', 12, 'bold'), foreground='magenta')  # Button style

encrypt_button = ttk.Button(frame, text="ENCRYPT", command=encrypt_message, style='TButton')
encrypt_button.pack()

decrypt_button = ttk.Button(frame, text="DECRYPT", command=decrypt_message, style='TButton')
decrypt_button.pack()

result_text = tk.Text(frame, width=60, height=15)
result_text.pack()

# Generate RSA key pair
rsa_key = RSA.generate(2048)

root.mainloop()